In [2]:
from gurobipy import GRB, Model

# Transportation model

## Read data

In a real-world example you would read this data from files or a database using data processing packages like Pandas.

<img src="Data_Transportation.png">

Warehouses

In [8]:
warehouses = [f"W{i}" for i in range(1, 4)]

Customers

In [10]:
customers = [f"C{i}" for i in range(1, 6)]

Variable transportation costs (per unit)

In [13]:
transport_costs = {
    ('W1', 'C1'): 4,
    ('W1', 'C2'): 5,
    ('W1', 'C3'): 6,
    ('W1', 'C4'): 8,
    ('W1', 'C5'): 10,
    ('W2', 'C1'): 6,
    ('W2', 'C2'): 4,
    ('W2', 'C3'): 3,
    ('W2', 'C4'): 5,
    ('W2', 'C5'): 8,
    ('W3', 'C1'): 9,
    ('W3', 'C2'): 7,
    ('W3', 'C3'): 4,
    ('W3', 'C4'): 2,
    ('W3', 'C5'): 4,
    
}

Demand

In [15]:
dem = {
    'C1': 80,
    'C2': 270,
    'C3': 250,
    'C4': 160,
    'C5': 180
}

Capacity

In [16]:
cap = {
    'W1': 500,
    'W2': 500,
    'W3': 500
}

## Optimization Model

In [17]:
m = Model('Transportation model')

Using license file c:\gurobi\gurobi.lic
Set parameter CloudAccessID
Set parameter CloudSecretKey
Set parameter CloudPool to value 427039-DSM
Waiting for cloud server to start (pool 427039-DSM)...
Starting...
Starting...
Starting...
Starting...
Starting...
Compute Server job ID: 6310fc22-36b6-4d25-9074-39eee9e9f278
Capacity available on '427039-DSM' cloud pool - connecting...
Established HTTPS encrypted connection


### Decision variables

How much to transport from which warehouse to which customer? (we cannot transport a negative amount -> lower bound = 0)

In [18]:
x = m.addVars(warehouses, customers, lb=0)

### Constraints

Satisfy demand, i.e. for EACH customer we consider the aggregated transportation flows from all warehouses.

In [20]:
m.addConstrs(
    x.sum('*', c) >= dem[c] for c in customers
)
m.update()

Respect maximal warehouse capacity

In [21]:
m.addConstrs(
    x.sum(w, '*') <= cap[w] for w in warehouses
)
m.update()

### Objective function

The total transportation costs (unit*cost) is to be minimized

In [22]:
m.setObjective(x.prod(transport_costs))

### Solve the problem

In [23]:
m.optimize()

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (linux64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 8 rows, 15 columns and 30 nonzeros
Model fingerprint: 0x6ddedd56
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 1e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [8e+01, 5e+02]
Presolve time: 0.01s
Presolved: 8 rows, 15 columns, 30 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   9.400000e+02   0.000000e+00      0s
       6    3.2100000e+03   0.000000e+00   0.000000e+00      0s

Solved in 6 iterations and 0.02 seconds
Optimal objective  3.210000000e+03


### Get the solution

In [25]:
for w in warehouses:
    for c in customers:
        print(f'From warehouse {w} to customer {c}: {x[w, c].X}')
       
print(f'The total transportation costs are {m.ObjVal}')

From warehouse W1 to customer C1: 80.0
From warehouse W1 to customer C2: 20.0
From warehouse W1 to customer C3: 0.0
From warehouse W1 to customer C4: 0.0
From warehouse W1 to customer C5: 0.0
From warehouse W2 to customer C1: 0.0
From warehouse W2 to customer C2: 250.0
From warehouse W2 to customer C3: 250.0
From warehouse W2 to customer C4: 0.0
From warehouse W2 to customer C5: 0.0
From warehouse W3 to customer C1: 0.0
From warehouse W3 to customer C2: 0.0
From warehouse W3 to customer C3: 0.0
From warehouse W3 to customer C4: 160.0
From warehouse W3 to customer C5: 180.0
The total transportation costs are 3210.0


### Summary

The whole optimization problem consists of just these 6 lines of code

In [18]:
m = Model('Transportation model')
x = m.addVars(warehouses, customers, lb=0)
m.addConstrs(x.sum('*', c) >= dem[c] for c in customers)
m.addConstrs(x.sum(w, '*') <= cap[w] for w in warehouses)
m.setObjective(x.prod(transport_costs))
m.optimize()

Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (win64)
Optimize a model with 8 rows, 15 columns and 30 nonzeros
Model fingerprint: 0x6ddedd56
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 1e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [8e+01, 5e+02]
Presolve time: 0.00s
Presolved: 8 rows, 15 columns, 30 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   9.400000e+02   0.000000e+00      0s
       6    3.2100000e+03   0.000000e+00   0.000000e+00      0s

Solved in 6 iterations and 0.00 seconds
Optimal objective  3.210000000e+03
